In [1]:
#　モーニングスター社のWebサイトから複数の投資信託の基準価格データを取得し、ひとつのEXCELファイルにまとめて出力する。 
#  このプログラムを使用したことによる損害や損失などの一切の責任は負いません。自己責任でお使いください。
#　Browser:Chrome
#
import os
import time
import pandas as pd
import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

# データ出力先のフォルダをdownloadsに設定する（ユーザの環境などの事情により変える）
os.chdir('C://users//myaccount//downloads')

# MorningStar社のWebサイトから基準価額のデータをダウンロードする関数

def MorningStar_web(fund_name,file_name_short,fund_id,start_ymd,end_ymd):

  options = webdriver.ChromeOptions()
  print('connectiong to remote browser...')
  driver = webdriver.Chrome(options=options)

  driver.get('https://www.morningstar.co.jp/FundData/Download.do?fnc='+fund_id)

#データ取得する開始日、終了日
  start_y = start_ymd[0:4]
  if start_ymd[4] == '0':
      start_m = start_ymd[5]
  else:
      start_m = start_ymd[4:6]

  if start_ymd[6] == '0':
      start_d = start_ymd[7]
  else:
      start_d = start_ymd[6:8]

  end_y = end_ymd[0:4]
  if end_ymd[4] == '0':
      end_m = end_ymd[5]
  else:
      end_m = end_ymd[4:6]

  if end_ymd[6] == '0':
      end_d = end_ymd[7]
  else:
      end_d = end_ymd[6:8]

#ドロップダウンの要素をname属性値から取得
  dropdown_From_Y = driver.find_element(By.NAME,"selectStdYearFrom")
  dropdown_From_M = driver.find_element(By.NAME,"selectStdMonthFrom")
  dropdown_From_D = driver.find_element(By.NAME,"selectStdDayFrom")

  dropdown_To_Y = driver.find_element(By.NAME,"selectStdYearTo")
  dropdown_To_M = driver.find_element(By.NAME,"selectStdMonthTo")
  dropdown_To_D = driver.find_element(By.NAME,"selectStdDayTo")

#開始日のドロップダウンの要素を選択し、開始日を設定
  select = Select(dropdown_From_Y)
  select.select_by_value(start_y)

  select = Select(dropdown_From_M)
  select.select_by_value(start_m)

  select = Select(dropdown_From_D)
  select.select_by_value(start_d)

#終了日のドロップダウンの要素を選択し、終了日を設定
  select = Select(dropdown_To_Y)
  select.select_by_value(end_y)

  select = Select(dropdown_To_M)
  select.select_by_value(end_m)

  select = Select(dropdown_To_D)
  select.select_by_value(end_d)

# ダウンロードボタンを押下
  driver.find_element(By.XPATH,"//div[@class=\"dlboxr\"]/p/input").click()

# ダウンロードのため5秒間待つ
  time.sleep(5)
 
  original_file_nam = "基準価額_日次ベース_" + start_ymd + "-" + end_ymd +".csv"
  new_file_nam = file_name_short + "_" + start_ymd + "-" + end_ymd +".csv"

  os.rename(original_file_nam,new_file_nam)
    
  driver.quit()

In [2]:
#基準価額を取得する開始日、終了日を設定する

start_ymd = '20210401'
end_ymd =   '20220930'

#基準価額を取得する投資信託のリスト
fund_list =[ \
['三井住友　日本株式インデックス年金ファンド','SMBC日本株式INDEX','2003022101'],\
['ABグローバル・グロース・オポチュニティーズ（年金向け）','ABGlobalGrowth','2001112601'],\
['eMAXIS Slim先進国株式インデックス','eMAX先進株イ','2017022703'],\
['eMAXIS Slim国内株式（ＴＯＰＩＸ）','eMAXTOPIX','2017022705']\
]

for n in fund_list:
  MorningStar_web(n[0],n[1],n[2],start_ymd,end_ymd)


connectiong to remote browser...
connectiong to remote browser...
connectiong to remote browser...
connectiong to remote browser...


In [3]:
# Pandas にてCSVデータをひとつにまとめる

data_each1 = fund_list[0][1] + "_" + start_ymd + "-" + end_ymd +".csv"
df1 = pd.read_csv(data_each1,encoding="shift-jis")
df1 = df1.rename(columns={'基準価額': fund_list[0][1]})

if (len(fund_list) >= 2):
  data_each2 = fund_list[1][1] + "_" + start_ymd + "-" + end_ymd +".csv"
  df2 = pd.read_csv(data_each2,encoding="shift-jis")
  df2 = df2.rename(columns={'基準価額': fund_list[1][1]})

  dfw = pd.merge(df1,df2)
    
  for v in range(len(fund_list)-2):
     w = v + 2
     data_each = fund_list[w][1] + "_" + start_ymd + "-" + end_ymd +".csv"
     dfn = pd.read_csv(data_each,encoding="shift-jis")
     dfn = dfn.rename(columns={'基準価額': fund_list[w][1]})
     dfw = pd.merge(dfw,dfn)
else:
  dfw = df1
    
#日付書式の変換
for v in range(len(dfw.index)):
    date_old = str(dfw.iloc[v,0])
    dfw.iloc[v,0] = date_old[0:4]+'/'+date_old[4:6]+'/'+date_old[6:8]

# EXCEL として出力
excel_fname = "Compiled" + "_" + start_ymd + "-" + end_ymd +".xlsx"
dfw.to_excel(excel_fname)
